In [1]:
import pandas as pd
import clickhouse_connect as ch

In [2]:
# Подключение к серверу ClickHouse
client = ch.get_client(host='192.168.1.7', port=8123, username='default')


In [3]:
# Версия ClickHouse
version = client.query('SELECT version()')
print(f''' Версия ClickHouse: {version.result_rows}''')


 Версия ClickHouse: [('24.8.4.13',)]


# Cоздание таблиц

In [5]:
# Создание таблицы users 
creat_users ='''
CREATE TABLE default.users
(
    id UInt64,                        -- Поле id, которое используется для уникальной идентификации пользователя (целое беззнаковое 64-битное число)
    name String,                      -- Поле name для хранения имени пользователя (строка)
    email String,                     -- Поле email для хранения электронной почты пользователя (строка)
    created_at DateTime               -- Поле created_at для хранения времени создания записи (формат DateTime)
) 
ENGINE = MergeTree()                   -- Используем движок MergeTree, оптимизированный для больших объемов данных и аналитики
ORDER BY (created_at, id)              -- Определяем порядок хранения данных: сначала по дате создания, затем по идентификатору пользователя

'''

# Выполнение запроса
result = client.query(creat_users)


In [6]:
# Создание таблицы users 
creat_orders ='''
CREATE TABLE default.orders
(
    id UInt64,                        -- Уникальный идентификатор заказа (целое беззнаковое 64-битное число)
    user_id UInt64,                   -- Идентификатор пользователя, который сделал заказ (ссылка на таблицу users)
    total_price Float64,              -- Общая стоимость заказа (вещественное число с двойной точностью)
    created_at DateTime               -- Время создания заказа (формат DateTime)
) 
ENGINE = MergeTree()                   -- Используем движок MergeTree
ORDER BY (created_at, id)              -- Определяем порядок хранения данных: сначала по дате создания заказа, затем по идентификатору заказа

'''

# Выполнение запроса
result = client.query(creat_orders)

In [7]:
creat_order_items = '''
CREATE TABLE default.order_items
(   
    id UInt64,                        -- Уникальный идентификатор записи о товаре в заказе (целое беззнаковое 64-битное число)
    order_id UInt64,                  -- Идентификатор заказа, к которому относится этот товар (ссылка на таблицу orders)
    product_name String,              -- Название товара (строка)
    price Float64,                    -- Цена одного товара (вещественное число с двойной точностью)
    quantity UInt8                    -- Количество данного товара в заказе (целое 8-битное число)
) 
ENGINE = MergeTree()                   -- Движок MergeTree
ORDER BY id                            -- Сортировка данных по уникальному идентификатору записи (id)

'''
# Выполнение запроса
result = client.query(creat_order_items)

# 2. Генерация данных

In [9]:
users_data = '''
INSERT INTO users
-- Вставляем данные в таблицу users с использованием SELECT запроса

SELECT 
    number AS id,                                   -- Используем последовательное число из функции numbers() как уникальный идентификатор (id)
    concat('user_', toString(number)) AS name,       -- Генерируем имя пользователя в формате 'user_номер', где номер — это текущее значение из numbers()
    concat('user_', toString(number), '@example.com') AS email,  -- Создаем email в формате 'user_номер@example.com'
    addDays(now(), -rand()%1800) AS created_at      -- Устанавливаем дату создания учетной записи как случайную дату в прошлом (от текущей даты на случайное количество дней до 1800)
FROM 
    numbers(2000000)                                 -- Генерируем 2 миллиона строк с числами от 0 до 1 999 999

'''
# Выполнение запроса
result = client.query(users_data)

In [10]:
orders_data = '''

INSERT INTO orders
-- Вставляем данные в таблицу orders с использованием SELECT запроса

SELECT 
    number AS id,                                     -- Используем последовательное число из функции numbers() как уникальный идентификатор заказа (id)
    
    intDiv(number, rand() % 2000000 + 1) % 2000000 + 1 AS user_id,  
    -- Генерируем случайный идентификатор пользователя (user_id) от 1 до 2 миллионов. 
    -- Для этого берем целочисленное деление `number` на случайное число от 1 до 2 миллионов, затем берем остаток от деления на 2 миллиона и добавляем 1.
    
    round((rand() % 1500) + 50, 2) AS total_price,    
    -- Генерируем случайную сумму заказа (total_price) от 50 до 1550, округленную до двух знаков после запятой. 
    -- Используем функцию `rand()` для получения случайного числа, затем добавляем минимальное значение 50.

    addDays(now(), -rand()%1800) AS created_at      
    -- Создаем случайную дату создания заказа за последние 1800 дней (примерно 5 лет).
    -- Используем функцию `addDays()` для вычитания случайного количества дней от текущей даты.
    
FROM 
    numbers(1000000)                                  -- Генерируем 1 миллион строк с числами от 0 до 999 999

'''
# Выполнение запроса
result = client.query(orders_data)

In [11]:
order_items_data = '''
INSERT INTO default.order_items
-- Вставляем данные в таблицу order_items с использованием SELECT запроса

SELECT 
    number AS id,                                  -- Используем последовательное число из функции numbers() как уникальный идентификатор товара в заказе (id)
    
    intDiv(number, 3) AS order_id,                 -- Генерируем идентификатор заказа (order_id) для каждого товара. Каждому заказу присваивается по 3 товара.
    
    concat('product_', toString(rand()%100)) AS product_name, 
    -- Генерируем название товара в формате 'product_номер', где номер — это случайное число от 0 до 99.
    
    rand()%100 + 20 AS price,                      -- Генерируем случайную цену товара в диапазоне от 20 до 120, используя функцию rand() для случайного числа от 0 до 99, затем добавляем 20.

    rand()%5 + 1 AS quantity                       -- Генерируем случайное количество товара от 1 до 5, используя функцию rand() для получения случайного числа от 0 до 4, затем добавляем 1.
    
FROM 
    numbers(3000000)                               -- Генерируем 3 миллиона строк с числами от 0 до 2 999 999, по три товара на каждый заказ

'''
# Выполнение запроса
result = client.query(order_items_data)

Теперь, чтобы total_price совпадал с данными из таблицв order_items
я посчитаю total_price в таблице order_items
и создам новую таблицу orders с новым данными total_price

In [13]:
new_orders = '''
-- Создаем временную таблицу orders_new с пересчитанным total_price
CREATE TABLE default.orders_new
ENGINE = MergeTree()  -- Указываем движок таблицы
ORDER BY id  -- Определяем порядок сортировки данных в таблице по полю id
AS
SELECT
    l.id,  -- Идентификатор заказа
    l.user_id,  -- Идентификатор пользователя, сделавшего заказ
    sum(r.price * r.quantity) AS total_price,  -- Пересчитанная сумма заказа на основе цены и количества товаров
    l.created_at  -- Дата создания заказа
FROM default.orders as l
LEFT JOIN default.order_items r ON l.id = r.order_id  -- Объединяем таблицу заказов с таблицей позиций заказа по id заказа
GROUP BY l.id, l.user_id, l.created_at  -- Группируем результаты по id заказа, идентификатору пользователя и дате создания заказа

'''
# Выполнение запроса
result = client.query(new_orders)

In [14]:
drop_orders = '''
-- Удаляем старую таблицу orders
DROP TABLE default.orders
'''
# Выполнение запроса
result = client.query(drop_orders)

In [15]:
rename_orders = '''
-- Переименовываем новую таблицу в orders
RENAME TABLE default.orders_new TO default.orders
'''

# Выполнение запроса
result = client.query(rename_orders)

# 1.	Найти общее количество заказов каждого пользователя, который сделал более 10 заказов.

In [17]:
task1 = '''
SELECT
  COUNT(id) as number_of_orders,
  user_id
FROM
  default.orders
GROUP BY
  user_id
HAVING
  number_of_orders > 10
ORDER BY number_of_orders DESC
'''
# В таблице default.orders группируем по id  пользователям и подсчитываем для каждого пользователя, количество заказов. 
# После фильтруем пользователей с более чем 10 заказами и выводим в результат, отсортированный по количеству заказов от большего к меньшему.

# Выполнение запроса
result = client.query(task1)



In [18]:
df_1 = pd.DataFrame(result.result_rows, columns=result.column_names)
df_1.head()

,number_of_orders,user_id
0,749943,1
1,125130,2
2,41641,3
3,20591,4
4,12504,5


# 2.	Найти средний размер заказа для каждого пользователя за последний месяц.


Комментарий по метрике "Размер заказа":

В данном контексте "размер заказа" может интерпретироваться двумя различными способами:

Итоговая цена заказа (total_price) — это сумма стоимости всех товаров в заказе, что отражает общий денежный объем, потраченный пользователем на один заказ.

Количество товаров в заказе — это общее количество единиц товара, содержащихся в заказе, которое отражает объем закупленных товаров, но не учитывает стоимость.

Так как формулировка метрики не определена однозначно, требуется уточнение, какую именно метрику необходимо рассчитывать: итоговую стоимость заказа или количество товаров.

В связи с этим, приведены два примера расчетов для каждой из метрик:

Пример 1: Рассчет среднего размера заказа по количеству товаров.

Пример 2: Рассчет среднего размера заказа по итоговой цене (total_price).


### Описание запроса task2_1:

1. В таблице `order_items` подсчитываем среднее количество товаров (сумма товаров в заказах деленная на количество заказов) для каждого пользователя. 
Для этого объединяем таблицы `order_items` и `orders` по идентификатору заказа. 
2. Далее фильтруем заказы, созданные за последний месяц (за последние 30 дней от текущей даты), и группируем по пользователям.

In [29]:
# Рассчет среднего размера заказа по количетсву товаров 
task2_1 = '''
SELECT
  SUM(l.quantity) / COUNT(r.id) as AVG_size,
  r.user_id
FROM
  default.order_items as l
INNER JOIN
  default.orders as r ON l.order_id = r.id
WHERE
  r.created_at >= dateSub(MONTH, 1, now()) -- за последний месяц ( т.е с 16.09.24 по 16.08.24)
GROUP BY
  r.user_id
  '''

# Комментарий
# Если необходимо получить отчет за август, то необходимо в фильтрации использовать:
# WHERE r.created_at >= toStartOfMonth(addMonths(today(), -1))  -- начало предыдущего месяца
# AND r.created_at < toStartOfMonth(today())  -- начало текущего месяца

# Выполнение запроса
result = client.query(task2_1)

In [31]:
df_2_1 = pd.DataFrame(result.result_rows, columns=result.column_names)
df_2_1.head()

,AVG_size,user_id
0,3.666667,198
1,2.833333,66
2,3.000000,94
3,3.000000,69
4,2.000000,89


### Описание запроса task_alt:

1. **Вложенный запрос** группирует заказы по идентификатору заказа (`order_id`) и вычисляет сумму товаров в каждом заказе (`quantity_sum`).
2. **Внешний запрос** затем соединяет результаты вложенного запроса с таблицей заказов `orders` по полю `order_id`.
3. Запрос фильтрует заказы, созданные за последний месяц.
4. После этого для каждого пользователя вычисляется средний размер заказа, где средний размер — это сумма товаров по всем заказам пользователя, делённая на количество заказов.
5. В итоге результаты группируются по пользователям и выводятся с сортировкой по среднему размеру заказа.

Данный запрос альтернативный, так как фильтрация происходит не сразу и этим самым производодительность ниже, а так же присутствут избыточные шаги. 

In [35]:
# Альтернативный запрос 
task_alt ='''
SELECT
  SUM(quantity_sum) / COUNT(order_id) as AVG_size,
  user_id
FROM
  (
    SELECT
      SUM(quantity) as quantity_sum,
      order_id
    FROM
      default.order_items
    GROUP BY
      order_id
  ) as l
  INNER JOIN default.orders as r ON l.order_id = r.id
WHERE
  created_at >= dateSub(MONTH, 1, now())
GROUP BY
  user_id
'''

# Выполнение запроса
result = client.query(task_alt)

In [37]:
df_2_alt = pd.DataFrame(result.result_rows, columns=result.column_names)
df_2_alt.head()

,AVG_size,user_id
0,8.0,161
1,9.0,69
2,14.0,320
3,7.5,30
4,10.0,532


### Описание запроса task2:

В таблице `orders` подсчитываем среднюю сумму заказа для каждого пользователя. Для этого используем агрегатную функцию `AVG()` по полю `total_price`, которая вычисляет среднее значение стоимости заказа.

Далее фильтруем заказы, созданные за последний месяц (за последние 30 дней от текущей даты), и группируем данные по пользователям.

In [41]:
# Рассчет среднего размера заказа по итоговой цене 
task2 = '''
SELECT
  user_id,
  AVG(total_price) as AVG_price
FROM
  default.orders
WHERE
  created_at >= dateSub(MONTH, 1, now()) -- за последний месяц (т.е с 16.09.24 по 16.08.24)
GROUP BY
  user_id
'''
# Выполнение запроса
result = client.query(task2)


In [43]:
df_2 = pd.DataFrame(result.result_rows, columns=result.column_names)
df_2.head()

,user_id,AVG_price
0,198,674.0
1,66,596.0
2,94,624.0
3,69,744.0
4,89,363.0


# 3.	Найти средний размер заказа за каждый месяц в текущем году и сравнить его с средним размером заказа за соответствующий месяц в прошлом году

Так же как и для второго задания рассчитаю два варианта. 

### Описание запроса task3_1:
Сначала вычисляется средний размер заказа за каждый месяц прошлого и текущего года.

1. В подзапросе `last_year_data` для таблиц `order_items` и `orders` рассчитывается средний размер заказа за каждый месяц прошлого года. Для этого производится группировка по месяцам с помощью функции `toMonth()`, а средний размер заказа вычисляется как отношение суммы количества товаров в заказах к количеству заказов за каждый месяц. Затем фильтруются данные по предыдущему году с использованием функции `toYear()`.

2. В подзапросе `this_year_data` аналогично вычисляется средний размер заказа за каждый месяц текущего года. Агрегация проводится по месяцам с использованием тех же методов, что и для прошлого года, но данные фильтруются для текущего года.

3. Затем результаты обоих подзапросов объединяются с помощью операции `FULL OUTER JOIN` по полю месяца, чтобы вывести данные за оба года. Поле месяца выбирается с помощью функции `COALESCE()`, которая берет значение месяца из любого подзапроса, если его нет в другом.

4. В конечном запросе отображаются месяц, средний размер заказа за каждый месяц прошлого года и текущего года, а результат сортируется по месяцу.

In [45]:
task3_1 = '''
WITH 
-- Средний размер заказа за каждый месяц прошлого года
last_year_data AS (
    SELECT
        toMonth(r.created_at) as month,
        SUM(l.quantity) / COUNT(r.id) as AVG_size_last_year 
    FROM
        default.order_items as l
    INNER JOIN 
        default.orders as r ON l.order_id = r.id  
    WHERE
        toYear(r.created_at) = toYear(now()) - 1
    GROUP BY
        toMonth(r.created_at)
),

-- Средний размер заказа за каждый месяц текущего года
this_year_data AS (
    SELECT
        toMonth(r.created_at) as month,
        SUM(l.quantity) / COUNT(r.id) as AVG_size_this_year  
    FROM
        default.order_items as l
    INNER JOIN 
        default.orders as r ON l.order_id = r.id  
    WHERE
        toYear(r.created_at) = toYear(now())
    GROUP BY
        toMonth(r.created_at)
)

-- Объединение данных за прошлый и текущий годы
SELECT
    COALESCE(l.month, r.month) as month,
    l.AVG_size_last_year,
    r.AVG_size_this_year
FROM
    last_year_data as l
FULL OUTER JOIN 
    this_year_data as r ON l.month = r.month
ORDER BY
    month

  '''
result = client.query(task3_1)

In [47]:
df_3_1 = pd.DataFrame(result.result_rows, columns=result.column_names)
df_3_1.head()

,month,AVG_size_last_year,AVG_size_this_year
0,1,2.997861,2.999479
1,2,2.998664,2.991893
2,3,2.998840,2.993174
3,4,3.008468,3.003186
4,5,3.010733,2.995560


### Описание запроса task3_1:
Вначала вычисляется средняя сумма заказа за каждый месяц прошлого и текущего года.

1. В подзапросе `last_year` для таблицы `orders` рассчитывается средняя сумма заказа за каждый месяц прошлого года. Для этого используется функция `AVG()` по полю `total_price`, которая вычисляет среднюю стоимость заказа за каждый месяц. Данные фильтруются для предыдущего года с помощью функции `toYear()`, а группировка проводится по месяцам с использованием функции `toMonth()`.

2. В подзапросе `this_year` аналогичным образом рассчитывается средняя сумма заказа за каждый месяц текущего года. Используются те же функции для вычисления средней стоимости заказа, но данные фильтруются для текущего года.

3. В основном запросе результаты обоих подзапросов объединяются с помощью операции `FULL JOIN` по полю месяца, чтобы вывести данные за оба года. Используется функция `COALESCE()`, которая выбирает значение месяца из любого подзапроса, если оно отсутствует в другом.

4. В финальной выборке отображаются месяц, средняя сумма заказа за каждый месяц текущего года и прошлого года, а результат сортируется по месяцу.

In [49]:
task3_2 = '''
WITH last_year AS (

SELECT
      toMonth(created_at) as month,
      AVG(total_price) as AVG_price_last_year
    FROM
      default.orders
    WHERE
      toYear(created_at) = toYear(now()) - 1
    GROUP BY
      toMonth(created_at) 

), 

this_year AS ( 
SELECT
      toMonth(created_at) as month,
      AVG(total_price) as AVG_price_this_year
    FROM
      default.orders
    WHERE
      toYear(created_at) = toYear(now())
    GROUP BY
      toMonth(created_at) 
)


SELECT
  COALESCE(l.month, r.month) as month,
  r.AVG_price_this_year,
  l.AVG_price_last_year
FROM
    last_year as l
FULL JOIN
    this_year as r
ON l.month = r.month
ORDER BY
    month
  '''
result = client.query(task3_2)

In [51]:
df_3_2 = pd.DataFrame(result.result_rows, columns=result.column_names)
df_3_2.head()

,month,AVG_price_this_year,AVG_price_last_year
0,1,631.423549,631.449494
1,2,628.898364,630.040993
2,3,629.191844,632.084769
3,4,632.700926,635.577500
4,5,629.225575,633.928129


 Данный запрос альтернативный по следующим причинам:

1. **Использование `LEFT JOIN` вместо `FULL JOIN`**:
   Во втором запросе используется `LEFT JOIN`, что означает, что будут выбраны все месяцы из прошлого года, даже если в текущем году нет данных. Однако, если в текущем году есть данные, а в прошлом — нет, такие записи будут потеряны. 
2. **Лучшее использование `COALESCE()`**, которая позволяет корректно отображать месяцы даже если данные есть только за один год. Результат может содержать пробелы в данных за определенные месяцы.

3. А так же присутствут избыточные шаги

In [53]:
# Альтернативный запрос 
task3_alt = '''
SELECT
  month,
  AVG_price_this_year,
  AVG_price_last_year
FROM
  (
    SELECT
      toMonth(created_at) as month,
      AVG(total_price) as AVG_price_last_year
    FROM
      default.orders
    WHERE
      toYear(created_at) = toYear(now()) - 1
    GROUP BY
      toMonth(created_at)
  ) as l
  LEFT JOIN (
    SELECT
      toMonth(created_at) as month,
      AVG(total_price) as AVG_price_this_year
    FROM
      default.orders
    WHERE
      toYear(created_at) = toYear(now())
    GROUP BY
      toMonth(created_at) 
  ) as r ON l.month = r.month
  '''
result = client.query(task3_alt)

In [55]:
df_3 = pd.DataFrame(result.result_rows, columns=result.column_names)
df_3

,month,AVG_price_this_year,AVG_price_last_year
0,1,631.423549,631.449494
1,2,628.898364,630.040993
2,3,629.191844,632.084769
3,4,632.700926,635.577500
4,5,629.225575,633.928129
5,6,630.672652,634.174698
6,7,630.269151,631.714717
7,8,632.873563,630.053369
8,9,634.434817,629.750075
9,10,0.000000,633.745483


# 4. Найти 10 пользователей, у которых наибольшее количество заказов за последний год, и для каждого из них найти средний размер заказа за последний месяц.


### Описание запроса task4_1
В таблице `orders` выбираем 10 пользователей с наибольшим количеством заказов за текущий год. Для этого группируем заказы по пользователям и сортируем по количеству заказов, затем ограничиваем результат десятью записями.

Далее в таблице `order_items` подсчитываем среднее количество товаров (сумма товаров в заказах, деленная на количество заказов) для этих пользователей. Для этого объединяем таблицы `order_items` и `orders` по идентификатору заказа и фильтруем заказы, созданные за последний месяц (за последние 30 дней от текущей даты).

После этого фильтруем данные по пользователям из топ-10 и группируем результат по каждому пользователю.


In [57]:
task4_1 = '''
WITH top_users AS (
  SELECT
    user_id
  FROM
    default.orders
  WHERE
    toYear(created_at) = toYear(now())
  GROUP BY
    user_id
  ORDER BY
    COUNT(id) DESC
  LIMIT
    10
)
SELECT
  SUM(l.quantity) / COUNT(r.id) as AVG_size,
  r.user_id
FROM
  default.order_items as l
INNER JOIN
  (SELECT * FROM default.orders WHERE created_at >= dateSub(MONTH, 1, now())) as r
ON l.order_id = r.id
WHERE
  r.user_id IN (SELECT user_id FROM top_users)
GROUP BY
  r.user_id
  '''
result = client.query(task4_1)

In [59]:
df_4 = pd.DataFrame(result.result_rows, columns=result.column_names)
df_4

,AVG_size,user_id
0,3.007246,4
1,3.049383,3
2,3.000631,2
3,3.000000,5
4,3.011243,1
5,2.886076,6
6,3.002778,7
7,2.945578,9
8,3.092593,8
9,3.050314,10


 Данный запрос альтернативный по следующим причинам:

1. **Избыточная вложенность**: используется вложенный подзапрос для получения топ-10 пользователей, что добавляет лишний уровень вложенности.

2. **Перемещение условий фильтрации**: Фильтрация выполняется в основном запросе после подзапроса с пользователями, что приводит к дополнительной обработке данных.


In [63]:
task4_alt = '''
SELECT
  SUM(l.quantity) / COUNT(r.id) as AVG_size,
  r.user_id
FROM
  default.order_items as l
INNER JOIN default.orders as r ON l.order_id = r.id
WHERE
  r.created_at >= dateSub(MONTH, 1, now())
  AND r.user_id IN 
  (
    SELECT
      user_id
    FROM
      (
        SELECT
          COUNT(id) as count_orders,
          user_id
        FROM
          default.orders
        WHERE
          toYear(created_at) = toYear(now()) -- последний год
        GROUP BY
          user_id
        ORDER BY
          count_orders DESC
        LIMIT
          10
      )
    )
GROUP BY
  r.user_id
  '''
result = client.query(task4_1)


In [65]:
df_4_alt = pd.DataFrame(result.result_rows, columns=result.column_names)
df_4_alt.head()

,AVG_size,user_id
0,3.007246,4
1,3.049383,3
2,3.000631,2
3,3.000000,5
4,3.011243,1


### Описание запроса task4_2
В таблице `orders` выбираем 10 пользователей с наибольшим количеством заказов за текущий год. Для этого группируем данные по пользователям и сортируем результат по количеству заказов, ограничивая выборку десятью пользователями.

Далее для этих пользователей рассчитываем среднюю сумму заказа (`AVG(total_price)`) за последний месяц. Для этого фильтруем заказы, созданные за последние 30 дней от текущей даты, и группируем результат по каждому пользователю, попавшему в топ-10.

In [67]:
task4_2 = '''
WITH top_users AS (
  SELECT
    user_id
  FROM
    default.orders
  WHERE
    toYear(created_at) = toYear(now())
  GROUP BY
    user_id
  ORDER BY
    COUNT(id) DESC
  LIMIT
    10
)
SELECT 
    user_id,
    AVG(total_price) as AVG_price
FROM 
  default.orders

WHERE 
    created_at >= dateSub(MONTH, 1, now())
    AND user_id IN (SELECT user_id FROM top_users)
GROUP BY user_id
  '''
result = client.query(task4_2)

In [69]:
df_4_2 = pd.DataFrame(result.result_rows, columns=result.column_names)
df_4_2.head()

,user_id,AVG_price
0,4,645.711180
1,3,624.024691
2,2,637.575284
3,5,640.663968
4,1,633.323943
